In [12]:
import pandas as pd


def get_df(threshold=200):
    PATH = 's3://chess-puzzles/single-best-move/{year}-{month:0>2}/{shard}.csv'
    path = PATH.format(year=2013, month=1, shard=0)
    df = pd.read_csv(path)
    df = df[df['best_loss'] > threshold]
    return df


# df = get_df()
df = pd.read_csv('2013_1_0.csv')

In [50]:
import chess
import chess.engine
STOCKFISH_PATH = '../Stockfish/src/stockfish'


engine = chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH)



def stockfish_info(engine, fen, move=None, depth=10, multipv=None):
    board = chess.Board(fen)
    move = [chess.Move.from_uci(move)] if move else None
    return engine.analyse(
        board,
        root_moves=move,
        multipv=multipv,
        limit=chess.engine.Limit(depth=depth)
    )



class StockfishMovePVFeatures():

    def __init__(self, fen, move):
        self.board = chess.Board(fen)
        self.move = chess.Move.from_uci(move)
        self.board.push(self.move)
        info = stockfish_info(engine, self.board.fen(), depth=2, multipv=500)
        self.pvs = [pv['pv'] for pv in info]
        self.scores = [pv['score'] for pv in info]

In [84]:
import chess.svg
from IPython.display import display, HTML



def display_board(fen, move=None):
    board = chess.Board(fen)
    if move:
        move = chess.Move.from_uci(move)
    arrows = [chess.svg.Arrow(move.from_square, move.to_square)] if move else []
    return HTML(chess.svg.board(board, arrows=arrows, size=200, coordinates=False))


row = df.sample(1).iloc[0]
display_board(row['fen'], row['best_move'])

In [82]:
features = StockfishMovePVFeatures(row['fen'], row['best_move'])
[list(map(str, pv)) for pv in features.pvs]

[['c7d8', 'e1e8', 'd8e8', 'f7b7'],
 ['c7d6', 'c3b5', 'd6d5'],
 ['c7b8', 'e1e8', 'b7c8', 'f7g7'],
 ['e8e7', 'e1e7', 'c7b8', 'e7b7', 'b8c8'],
 ['c7c8', 'e1e8']]

In [80]:
features.scores

[PovScore(Cp(-60), WHITE),
 PovScore(Cp(-1334), WHITE),
 PovScore(Cp(-1412), WHITE),
 PovScore(Cp(-1414), WHITE),
 PovScore(Cp(-1438), WHITE),
 PovScore(Cp(-1838), WHITE),
 PovScore(Cp(-1863), WHITE),
 PovScore(Cp(-1864), WHITE),
 PovScore(Cp(-1866), WHITE),
 PovScore(Cp(-1866), WHITE),
 PovScore(Cp(-1868), WHITE),
 PovScore(Cp(-1869), WHITE),
 PovScore(Cp(-1871), WHITE),
 PovScore(Cp(-1871), WHITE),
 PovScore(Cp(-1872), WHITE),
 PovScore(Cp(-1877), WHITE),
 PovScore(Cp(-1879), WHITE),
 PovScore(Cp(-1915), WHITE),
 PovScore(Cp(-1920), WHITE),
 PovScore(Cp(-1934), WHITE),
 PovScore(Cp(-1934), WHITE),
 PovScore(Cp(-1936), WHITE),
 PovScore(Cp(-2000), WHITE),
 PovScore(Cp(-2443), WHITE)]